# Qwen2.5-1.5B Vanilla - Larger Baseline

**Platform**: Kaggle (GPU)
**Time**: ~1.5 hours for 817 questions

**Purpose**: Larger model baseline (4x SmolLM2) to compare against SmolLM2-360M + Minimax.

**Key comparison**: Can 360M + Minimax beat 1.5B vanilla on hallucination rate?

In [ ]:
!pip install -q transformers accelerate torch datasets google-genai

In [ ]:
# ====== CONFIGURATION ======
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
MODEL_NAME = "Qwen2.5-1.5B"
OUTPUT_FILE = "mc_results_qwen_vanilla.json"
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"  # <-- REPLACE THIS

In [ ]:
import json, torch, numpy as np
from tqdm import tqdm
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

In [ ]:
# Initialize Gemini for letter extraction
import os
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
from google import genai
gemini_client = genai.Client(api_key=GEMINI_API_KEY)
print("Gemini initialized for letter extraction")

In [ ]:
print(f"Loading {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16, device_map="auto")
model.eval()
print("Loaded!")

In [ ]:
# ====== CORE FUNCTIONS ======

def get_log_probs(prompt, completion):
    full = prompt + completion
    p_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    f_ids = tokenizer.encode(full, return_tensors="pt").to(device)
    with torch.no_grad(): logits = model(f_ids).logits
    p_len = p_ids.shape[1]
    if p_len >= f_ids.shape[1]: return float('-inf')
    c_logits = logits[0, p_len-1:-1, :]
    c_ids = f_ids[0, p_len:]
    lp = torch.log_softmax(c_logits, dim=-1)
    return lp.gather(1, c_ids.unsqueeze(1)).squeeze(1).sum().item()


def extract_letter_with_gemini(response: str, choices: list) -> tuple:
    """Use Gemini to extract which letter the model chose."""
    choice_text = "\n".join([f"{chr(65+i)}. {c}" for i, c in enumerate(choices)])
    
    prompt = f"""The model was asked to pick an answer from these options:
{choice_text}

The model responded: "{response}"

Which option (A, B, C, D, E, or F) did the model choose? 
Return ONLY a JSON object:
{{"letter": "A" or "B" or "C" etc}}"""

    try:
        resp = gemini_client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        text = resp.text.strip()
        
        if text.startswith("```"):
            text = text.split("```")[1]
            if text.startswith("json"):
                text = text[4:]
        text = text.strip()
        
        result = json.loads(text)
        letter = result.get("letter", "A").upper()
        if letter in [chr(65+i) for i in range(len(choices))]:
            return letter, ord(letter) - 65
        return "A", 0
    except:
        return "A", 0


def eval_mc1(row):
    q = row["question"]
    choices, labels = row["mc1_targets"]["choices"], row["mc1_targets"]["labels"]
    correct_idx = labels.index(1)
    
    choice_text = "\n".join([f"{chr(65+i)}. {c}" for i,c in enumerate(choices)])
    prompt = f"Question: {q}\n\nOptions:\n{choice_text}\n\nAnswer:"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        out = model.generate(inputs.input_ids, max_new_tokens=10, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    resp = tokenizer.decode(out[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()
    letter, idx = extract_letter_with_gemini(resp, choices)
    
    return {
        "correct": idx == correct_idx, 
        "chosen_idx": idx, 
        "chosen_letter": letter,
        "correct_idx": correct_idx,
        "correct_letter": chr(65 + correct_idx),
        "raw": resp
    }


def eval_mc2(row):
    q, mc2 = row["question"], row["mc2_targets"]
    prompt = f"Question: {q}\nAnswer:"
    correct = [np.exp(get_log_probs(prompt, " "+c)) for c,l in zip(mc2["choices"], mc2["labels"]) if l==1][:5]
    incorrect = [np.exp(get_log_probs(prompt, " "+c)) for c,l in zip(mc2["choices"], mc2["labels"]) if l==0][:5]
    t = sum(correct) + sum(incorrect)
    return sum(correct)/t if t > 0 else 0


def eval_mc3(row):
    q, mc2 = row["question"], row["mc2_targets"]
    prompt = f"Question: {q}\nAnswer:"
    c_lps = [get_log_probs(prompt, " "+c) for c,l in zip(mc2["choices"], mc2["labels"]) if l==1][:3]
    i_lps = [get_log_probs(prompt, " "+c) for c,l in zip(mc2["choices"], mc2["labels"]) if l==0][:3]
    wins = sum(1 for c in c_lps for i in i_lps if c > i)
    t = len(c_lps) * len(i_lps)
    return wins/t if t > 0 else 0

In [ ]:
print(f"Starting {MODEL_NAME} Vanilla...")
print(f"Start: {datetime.now()}")

results = []
for idx, row in enumerate(tqdm(data)):
    try:
        mc1 = eval_mc1(row)
        results.append({
            "idx": idx, 
            "question": row["question"],
            **mc1, 
            "mc2": eval_mc2(row), 
            "mc3": eval_mc3(row)
        })
        if (idx+1) % 50 == 0: 
            acc = sum(r['correct'] for r in results)/len(results)
            print(f"Progress {idx+1}/{len(data)}: MC1={acc:.3f}")
        if (idx+1) % 100 == 0:
            with open(f"checkpoint_{idx+1}.json", "w") as f:
                json.dump({"results": results}, f)
    except Exception as e:
        print(f"Error {idx}: {e}")
        results.append({"idx": idx, "correct": False, "mc2": 0, "mc3": 0})

print(f"End: {datetime.now()}")

In [ ]:
# ====== RESULTS ======
total = len(results)
correct_count = sum(r["correct"] for r in results)
wrong_count = total - correct_count

# For vanilla: every wrong answer = hallucination
hallucination_rate = wrong_count / total * 100
truthful_rate = correct_count / total * 100

mc1 = truthful_rate
mc2 = sum(r["mc2"] for r in results) / total * 100
mc3 = sum(r["mc3"] for r in results) / total * 100

print("\n" + "="*60)
print(f"RESULTS: {MODEL_NAME} Vanilla (No Verification)")
print("="*60)
print(f"\n--- KEY METRICS ---")
print(f"  Hallucination Rate:  {hallucination_rate:.1f}% ({wrong_count}/{total})")
print(f"  Truthful Rate:       {truthful_rate:.1f}% ({correct_count}/{total})")
print(f"  Abstention Rate:     0.0% (vanilla never abstains)")
print(f"\n--- MC SCORES ---")
print(f"  MC1: {mc1:.2f}%")
print(f"  MC2: {mc2:.2f}%")
print(f"  MC3: {mc3:.2f}%")
print("="*60)

# Save
output = {
    "model": MODEL_ID,
    "model_name": MODEL_NAME, 
    "method": "Vanilla",
    "total_questions": total,
    "summary": {
        "hallucination_rate": round(hallucination_rate, 2),
        "truthful_rate": round(truthful_rate, 2),
        "abstention_rate": 0.0,
    },
    "metrics": {"mc1": round(mc1, 2), "mc2": round(mc2, 2), "mc3": round(mc3, 2)},
    "results": results
}

with open(OUTPUT_FILE, "w") as f:
    json.dump(output, f, indent=2)

print(f"\nSaved to {OUTPUT_FILE}")
print(f"\nKey result: {MODEL_NAME} Vanilla has {hallucination_rate:.1f}% hallucination rate")
print(f"Compare this to SmolLM2-360M + Minimax hallucination rate!")